# Imports

In [1]:
import pandas as pd
import numpy as np
from recommender import load_data

# Loading & Inspecting Data

In [2]:
# load
df = load_data()

# get first 5
pd.set_option('display.max_columns', None)
print("df.head()=================================================")
print(df.head())

# get data structure
print("df.dtypes=================================================")
print(df.dtypes)

# get num of nulls
print("df.isnul().sum()=================================================")
print(df.isnull().sum())

# get num of uniques
print("df.nunique()=================================================")
print(df.nunique())

[INFO] CSV not found. Downloading from KaggleHub...


100%|██████████| 450M/450M [00:19<00:00, 24.3MB/s] 


[SUCCESS] CSV downloaded and saved to: c:\Users\wange\CS506_FinalProject\new-game-plus-backend\newgameplus\data\games.csv
df.head()=================================================
    appid                             name release_date  required_age  price  \
0     730                 Counter-Strike 2   2012-08-21             0   0.00   
1  578080              PUBG: BATTLEGROUNDS   2017-12-21             0   0.00   
2     570                           Dota 2   2013-07-09             0   0.00   
3  271590        Grand Theft Auto V Legacy   2015-04-13            17   0.00   
4  488824  Tom Clancy's Rainbow Six® Siege   2015-12-01            17  19.99   

   dlc_count                               detailed_description  \
0          1  For over two decades, Counter-Strike has offer...   
1          0  LAND, LOOT, SURVIVE! Play PUBG: BATTLEGROUNDS ...   
2          2  The most-played game on Steam. Every day, mill...   
3          0  When a young street hustler, a retired bank ro...   
4  

# Preprocessing

In [36]:
# remove the null name rows
df = df.drop(df.loc[df['name'].isna()].index)

# drop USELESS columns
df = df.drop(['name','tags','reviews', 'appid', 'detailed_description', 'about_the_game', 'short_description', 'header_image', 'website', 'support_url','support_email','metacritic_url','notes', 'packages', 'developers', 'publishers','screenshots', 'movies','user_score','score_rank','estimated_owners','positive','negative'], axis=1) # edw: undrop reviews and sentiment thing

In [ ]:
# clean data
df['release_date_cleaned'] = pd.to_datetime(df['release_date'], errors='coerce')
df = df.drop(['release_date'], axis=1)




required_age                                                                0
price                                                                     0.0
dlc_count                                                                   1
windows                                                                  True
mac                                                                     False
linux                                                                    True
metacritic_score                                                            0
achievements                                                                1
recommendations                                                       4401572
supported_languages         ['Czech', 'Danish', 'Dutch', 'English', 'Finni...
full_audio_languages                                ['English', 'Indonesian']
categories                  ['Multi-player', 'Cross-Platform Multiplayer',...
genres                                             ['Action', 'F

In [ ]:
#Identifying the unique genres
import ast
df['genres'] = df['genres'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

unique_genres = set()

for genres in df['genres']:
    if isinstance(genres, list):
        unique_genres.update(genres)

unique_genre_list = list(unique_genres)
print("Unique genres:", unique_genre_list)

Unique genres: ['Photo Editing', 'Action', 'Simulation', 'Nudity', 'Racing', 'Short', 'Indie', 'Gore', '360 Video', 'Episodic', 'Violent', 'Adventure', 'Accounting', 'RPG', 'Early Access', 'Strategy', 'Design & Illustration', 'Education', 'Audio Production', 'Sexual Content', 'Sports', 'Animation & Modeling', 'Movie', 'Documentary', 'Utilities', 'Casual', 'Game Development', 'Web Publishing', 'Free To Play', 'Tutorial', 'Software Training', 'Massively Multiplayer', 'Video Production']


In [ ]:
#One hot encoding the genres
for genre in unique_genres:
    df[genre] = df['genres'].apply(lambda x: 1 if genre in x else 0)

print(df.head())

    appid                             name release_date  required_age  price  \
0     730                 Counter-Strike 2   2012-08-21             0   0.00   
1  578080              PUBG: BATTLEGROUNDS   2017-12-21             0   0.00   
2     570                           Dota 2   2013-07-09             0   0.00   
3  271590        Grand Theft Auto V Legacy   2015-04-13            17   0.00   
4  488824  Tom Clancy's Rainbow Six® Siege   2015-12-01            17  19.99   

   dlc_count                               detailed_description  \
0          1  For over two decades, Counter-Strike has offer...   
1          0  LAND, LOOT, SURVIVE! Play PUBG: BATTLEGROUNDS ...   
2          2  The most-played game on Steam. Every day, mill...   
3          0  When a young street hustler, a retired bank ro...   
4          9  Edition Comparison Ultimate Edition The Tom Cl...   

                                      about_the_game  \
0  For over two decades, Counter-Strike has offer...   
1  L